### Import Functions

In [49]:
from ODESystem_Chp1 import dNdt
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd

### Function to Import Pollutant v. Ecosystem Component Associations

In [50]:
def load_df(xμC, xrM, xrT, xαCT, xαMT,
            xhB, xhG, xhS, xeB, xeG, xeS):

    base_path = (
        "C:/Users/gails/Desktop/Team Sediment/"
        "Chapter 1 - Theory/Current Analyses/"
        "Adding Nutrient & Sediment Associations/"
    )
    
    
    μC_st = pd.read_pickle(f"{base_path}μC_strength{xμC}")
    rM_st = pd.read_pickle(f"{base_path}rM_strength{xrM}")
    rT_st = pd.read_pickle(f"{base_path}rT_strength{xrT}")
    
    αCT_st = pd.read_pickle(f"{base_path}αCT_strength{xαCT}")
    αMT_st = pd.read_pickle(f"{base_path}αMT_strength{xαMT}")
    
    hB_st = pd.read_pickle(f"{base_path}hB_strength{xhB}")
    hG_st = pd.read_pickle(f"{base_path}hG_strength{xhG}")
    hS_st = pd.read_pickle(f"{base_path}hS_strength{xhS}")
    
    eB_st = pd.read_pickle(f"{base_path}eB_strength{xeB}")
    eG_st = pd.read_pickle(f"{base_path}eG_strength{xeG}")
    eS_st = pd.read_pickle(f"{base_path}eS_strength{xeS}")


    return (
        μC_st, rM_st, rT_st,
        αCT_st, αMT_st,
        hB_st, hG_st, hS_st,
        eB_st, eG_st, eS_st
    )

### Function to Run ODE

In [51]:
def ODEfunction(
    scenario,
    run_id,
    dt,
    NUMSTEPS,
    NUMYEARS,
    time_points,   
    sed_value,
    nutr_value,
    init_cond,
    benthos,
    browser,
    grazer,
    scraper,
    null,
    strength,
    poll_df):

    
    #init_cond
    B0, G0, S0 = init_cond['B0'], init_cond['G0'], init_cond['S0']
    
    #benthos
    rC, αMC, μM, μT = benthos['rC'], benthos['αMC'], benthos['μM'], benthos['μT']
    
    #herbivores
    ηB, sB, σB, θB, dB, ρB, φB = browser['ηB'], browser['sB'], browser['σB'], browser['θB'], browser['dB'], browser['ρB'], browser['φB']
    ηG, sG, σG, θG, dG, ρG, φG = grazer['ηG'], grazer['sG'], grazer['σG'], grazer['θG'], grazer['dG'], grazer['ρG'], grazer['φG']
    ηS, sS, σS, θS, dS, ρS, φS = scraper['ηS'], scraper['sS'], scraper['σS'], scraper['θS'], scraper['dS'], scraper['ρS'], scraper['φS']
    h, η, s, σ, θ, e, d, ρ, φ  = null['h'], null['η'], null['s'], null['σ'], null['θ'], null['e'], null['d'], null['ρ'], null['φ']
    
    #strength
    xμC, xrM, xrT, xαCT, xαMT = strength['xμC'], strength['xrM'], strength['xrT'], strength['xαCT'], strength['xαMT']
    xhB, xhG, xhS, xeB, xeG, xeS = strength['xhB'], strength['xhG'], strength['xhS'], strength['xeB'], strength['xeG'], strength['xeS']
    
    #poll_df
    μC_st, rM_st, rT_st, αCT_st, αMT_st = poll_df['μC_st'], poll_df['rM_st'], poll_df['rT_st'], poll_df['αCT_st'], poll_df['αMT_st'],
    hB_st, hG_st, hS_st, eB_st, eG_st, eS_st = poll_df['hB_st'], poll_df['hG_st'], poll_df['hS_st'], poll_df['eB_st'], poll_df['eG_st'], poll_df['eS_st']                 
   

    outputs = []
    outputs_prime = []

    
    for df in [
        μC_st, rM_st, rT_st,
        αCT_st, αMT_st,
        hB_st, hG_st, hS_st,
        eB_st, eG_st, eS_st
    ]:
        df.set_index(["sed", "nutr"], inplace=True)


    for sed in sed_value:
    
        for nutr in nutr_value:
        
            for C0 in np.linspace(0.01,0.99,2):

                
                #Initial Conditions-----------------------------------------------------
                C0, M0, T0 = C0, ((1-C0)/2), ((1-C0)/2) 
                B0, G0, S0 = B0, G0, S0

                
                #Define Parameters Affected by Pollutants-----------------------------------------------------
                μC = μC_st.loc[(sed, nutr), "μC"]
                rM = rM_st.loc[(sed, nutr), "rM"]
                rT = rT_st.loc[(sed, nutr), "rT"]
                
                αCT = αCT_st.loc[(sed, nutr), "αCT"]
                αMT = αMT_st.loc[(sed, nutr), "αMT"]
                
                hB = hB_st.loc[(sed, nutr), "hB"]
                hG = hG_st.loc[(sed, nutr), "hG"]
                hS = hS_st.loc[(sed, nutr), "hS"]
                
                eB = eB_st.loc[(sed, nutr), "eB"]
                eG = eG_st.loc[(sed, nutr), "eG"]
                eS = eS_st.loc[(sed, nutr), "eS"]

                
                #Herbivores-----------------------------------------------------
                #Browsers
                if scenario == 'browser':
                    hB, ηB, sB, σB, θB, eB, dB, ρB, φB  = hB, ηB, sB, σB, θB, eB, dB, ρB, φB
                    hG, ηG, sG, σG, θG, eG, dG, ρG, φG  = h, η, s, σ, θ, e, d, ρ, φ
                    hS, ηS, sS, σS, θS, eS, dS, ρS, φS  = h, η, s, σ, θ, e, d, ρ, φ

                #Grazers
                elif scenario == 'grazer':
                    hG, ηG, sG, σG, θG, eG, dG, ρG, φG  = hG, ηG, sG, σG, θG, eG, dG, ρG, φG
                    hS, ηS, sS, σS, θS, eS, dS, ρS, φS  = h, η, s, σ, θ, e, d, ρ, φ
                    hB, ηB, sB, σB, θB, eB, dB, ρB, φB  = h, η, s, σ, θ, e, d, ρ, φ
                    
                #Scrapers
                elif scenario == 'scraper':
                    hS, ηS, sS, σS, θS, eS, dS, ρS, φS = hS, ηS, sS, σS, θS, eS, dS, ρS, φS
                    hB, ηB, sB, σB, θB, eB, dB, ρB, φB  = h, η, s, σ, θ, e, d, ρ, φ
                    hG, ηG, sG, σG, θG, eG, dG, ρG, φG  = h, η, s, σ, θ, e, d, ρ, φ

      
                #Browsers, Grazers, & Scrapers
                elif scenario == 'all_herbivores':
                    hB, ηB, sB, σB, θB, eB, dB, ρB, φB  = hB, ηB, sB, σB, θB, eB, dB, ρB, φB
                    hG, ηG, sG, σG, θG, eG, dG, ρG, φG  = hG, ηG, sG, σG, θG, eG, dG, ρG, φG
                    hS, ηS, sS, σS, θS, eS, dS, ρS, φS = hS, ηS, sS, σS, θS, eS, dS, ρS, φS

#make the ODE system so it's not positionally dependent!
                
                #Solve ODE
                sol = solve_ivp(dNdt, [0,NUMYEARS], [C0, M0, T0, B0, G0, S0], 
                                method = 'RK45', args = 
                                (rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                                     #benthos
                                ρB, hB, ηB, sB, φB, σB, θB, eB, dB,                                         #browsers
                                ρG, hG, ηG, sG, φG, σG, θG, eG, dG,                                         #grazers
                                ρS, hS, ηS, sS, φS, σS, θS, eS, dS, dt),                                    #scrapers
                                dense_output=True)
      
                N = sol.sol(time_points)
    
                C_array = N[0,:]
                M_array = N[1,:]
                T_array = N[2,:]
                
                B_array = N[3,:]
                G_array = N[4,:]
                S_array = N[5,:]


                #Categorize State at Equilibrium-----------------------------------------------------
                thr = 0.01

                #Benthos
                Cp = C_array[-1]
                Mp = M_array[-1]
                Tp = T_array[-1]
                Fp = 1 - (Cp + Mp + Tp)
                
                C_t = Cp >= thr
                M_t = Mp >= thr
                T_t = Fp >= thr

                if C_t and not M_t and not T_t:
                    ben_col = "allC"
                
                elif M_t and not C_t and not T_t:
                    ben_col = "allM"
                
                elif T_t and not C_t and not M_t:
                    ben_col = "allT"
                
                elif not C_t and not M_t and not T_t:
                    ben_col = "allF"
                
                else:
                    dom_val = max(Cp, Mp, Tp, Fp)
                
                    if dom_val == Cp:
                        ben_col = "mostlyC" if T_t else "mostlyC_noT"
                
                    elif dom_val == Mp:
                        ben_col = "mostlyM" if T_t else "mostlyM_noT"
                
                    elif dom_val == Tp:
                        ben_col = "mostlyT"
                    
                    elif dom_val == Fp:
                        ben_col = "mostlyF"
                        
                    else:
                        ben_col = "other"

                #Herbivores
                Bp = B_array[-1]
                Gp = G_array[-1]
                Sp = S_array[-1]

                B_t = Bp >= thr
                G_t = Gp >= thr
                S_t = Sp >= thr

                if B_t and not G_t and not S_t:
                    herb_col = "allB"
                
                elif G_t and not B_t and not S_t:
                    herb_col = "allG"
                
                elif S_t and not B_t and not G_t:
                    herb_col = "allS"
                
                elif not C_t and not M_t and not T_t:
                    herb_col = "nofish"

                else:
                    dom_val = max(Bp, Gp, Sp)
                
                    if dom_val == Bp:
                        herb_col = "mostlyB"
                
                    elif dom_val == Gp:
                        herb_col = "mostlyG"
                
                    elif dom_val == Sp:
                        herb_col = "mostlyS"

                    else:
                        herb_col = "other"

                
                #Ensure Run Reached Equilibrium-----------------------------------------------------
                Cp2 = C_array[-2]
                Mp2 = M_array[-2]
                Tp2 = T_array[-2]

                Bp2 = B_array[-2]
                Gp2 = G_array[-2]
                Sp2 = S_array[-2]                
               
                if abs(Cp - Cp2) < 0.00001 and abs(Mp - Mp2) < 0.00001 and abs(Tp - Tp2) < 0.00001 and abs(Bp - Bp2) < 0.00001 and abs(Gp - Gp2) < 0.00001 and abs(Sp - Sp2) < 0.00001:
                    equil = 'yes'

                else:
                    equil = 'no'

                #Save Results-----------------------------------------------------
                benthos = (rC, rM, rT, αCT, αMT, αMC, μC, μM, μT)
                browsers = (hB, ηB, sB, σB, θB, eB, dB, ρB, φB)
                grazers = (hG, ηG, sG, σG, θG, eG, dG, ρG, φG)
                scrapers = (hS, ηS, sS, σS, θS, eS, dS, ρS, φS)
                associations = (xμC, xrM, xrT, xαCT, xαMT, xhB, xhG, xhS, xeB, xeG, xeS, sed, nutr)
                state_eq = (Cp, Mp, Tp, Bp, Gp, Sp)
                labels = (ben_col, herb_col, equil)
                                    
                outputs.append(
                    benthos
                    + browsers
                    + grazers
                    + scrapers
                    + associations
                    + state_eq
                    + labels
                 )

                
                benthos_c = ('rC', 'rM', 'rT', 'αCT', 'αMT', 'αMC', 'μC', 'μM', 'μT')
                browsers_c = ('hB', 'ηB', 'sB', 'σB', 'θB', 'eB', 'dB', 'ρB', 'φB')
                grazers_c = ('hG', 'ηG', 'sG', 'σG', 'θG', 'eG', 'dG', 'ρG', 'φG')
                scrapers_c = ('hS', 'ηS', 'sS', 'σS', 'θS', 'eS', 'dS', 'ρS', 'φS')
                associations_c = ('xμC', 'xrM', 'xrT', 'xαCT', 'xαMT', 'xhB', 'xhG', 'xhS', 'xeB', 'xeG', 'xeS', 'sed', 'nutr')
                state_eq_c = ('Cp', 'Mp', 'Tp', 'Bp', 'Gp', 'Sp')
                labels_c = ('ben_col', 'herb_col', 'equil')

                columns = (
                    benthos_c
                    + browsers_c
                    + grazers_c
                    + scrapers_c
                    + associations_c
                    + state_eq_c
                    + labels_c
                )

        
    df_prime = pd.DataFrame(outputs_prime, columns=columns)
          
    return(df_prime)